# Worksheet 07

Name: Jiahang Li 
UID: U00295086

### Topics

- Soft Clustering
- Clustering Aggregation

### Probability Review

Read through [the following](https://medium.com/@gallettilance/overview-of-probability-3272b72c82c8)

### Soft Clustering

We generate 10 data points that come from a normal distribution with mean 5 and variance 1.

In [18]:
import random
import numpy as np
from sklearn.cluster import KMeans

mean = 5
stdev = 1

c1 = np.random.normal(mean, stdev, 10).tolist()
print(c1)

[5.040525465029493, 5.251256500625921, 3.9833619414461388, 4.482574259686124, 4.3205902411411365, 4.788609178832583, 5.58229254648858, 5.285518742776113, 5.255443871801707, 5.849085909039437]


a) Generate 10 more data points, this time coming from a normal distribution with mean 8 and variance 1.

In [19]:
c2 = np.random.normal(8, np.sqrt(1), 10).tolist()
print(c2)

[7.672892846969796, 8.440594605720708, 9.451596392202164, 8.526922903827446, 8.435372591529761, 7.94575907464858, 8.615242190106898, 7.19475829206822, 7.957020362100586, 9.094537366491185]


b) Flip a fair coin 10 times. If the coin lands on H, then pick the last data point of `c1` and remove it from `c1`, if T then pick the last data point from `c2` and remove it from `c2`. Add these 10 points to a list called `data`.

In [20]:
data = []
for i in range(10):
    # flip coin
    coin_output = random.choice([0, 1])
    if coin_output == 0:
        p1 = c1.pop()
        data.append(p1)
    else:
        p2 = c2.pop()
        data.append(p2)
print(data)

[5.849085909039437, 9.094537366491185, 5.255443871801707, 5.285518742776113, 7.957020362100586, 7.19475829206822, 5.58229254648858, 8.615242190106898, 7.94575907464858, 8.435372591529761]


c) This `data` is a Gaussian Mixture Distribution with 2 mixture components. Over the next few questions we will walk through the GMM algorithm to see if we can uncover the parameters we used to generate this data. First, please list all these parameters of the GMM that created `data` and the values we know they have.

Number of components (K): 2

P(c_j): [0.4, 0.6]
Mean : [5,8]
var: [1,1]

d) Let's assume there are two mixture components (note: we could plot the data and make the observation that there are two clusters). The EM algorithm asks us to start with a random `mean_j`, `variance_j`, `P(C_j)` for each component j. One method we could use to find sensible values for these is to apply K means with k=2 here.

1. the centroids would be the estimates of the `mean_j`
2. the intra-cluster variance could be the estimate of `variance_j`
3. the proportion of points in each cluster could be the estimate of `P(C_j)`

Go through this process and list the parameter estimates it gives. Are they close or far from the true values?

In [21]:
kmeans = KMeans(2, init='k-means++').fit(X=np.array(data).reshape(-1, 1))

c1 = [x[0] for x in filter(lambda x: x[1] == 0, zip(data, kmeans.labels_))]
print(c1)
c2 = [x[0] for x in filter(lambda x: x[1] == 1, zip(data, kmeans.labels_))]
print(c2)

prob_c = [ len(c1) / (len(c1) + len(c2)) , len(c2) / (len(c1) + len(c2)) ]
mean = [ sum(c1)/len(c1) , sum(c2)/len(c2) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2)  ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))

[5.849085909039437, 5.255443871801707, 5.285518742776113, 5.58229254648858]
[9.094537366491185, 7.957020362100586, 7.19475829206822, 8.615242190106898, 7.94575907464858, 8.435372591529761]
P(C_1) = 0.4,  P(C_2) = 0.6
mean_1 = 5.493085267526459,  mean_2 = 8.207114979490871
var_1 = 0.058562922634105466,  var_2 = 0.36031802327685664


Comparing these estimates to the true parameter values we have from the GMM, we can see that they are somewhat close but not exact. The mixing coefficients estimated by K-means are flipped but the values are close to the true values. The covariance matrix estimates from K-means are further off, with variances that are much smaller than the true variances of 1, and the variances are also different for the two components. Overall, using K-means to estimate the initial parameters is a useful strategy but not a guarantee that the estimates will be close to the true values.

e) For each data point, compute `P(C_j | X_i)`. Comment on which cluster you think each point belongs to based on the estimated probabilities. How does that compare to the truth?

In [22]:
from scipy.stats import norm

prob_c0_x = [] # P(C_j | X_i) = P(X_i | C_j) * P(C_j) / P(X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i) P(X_i) = sum_j P(X_i | C_j) * P(C_j)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0] * prob_c[0] / prob_x)
    prob_c1_x.append( pdf_i[1] * prob_c[1] / prob_x)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  5.849085909039437
probability of observing that point if it came from cluster 0 =  6.440618182944351e-08
probability of observing that point if it came from cluster 1 =  5.549747691806713e-10
point =  9.094537366491185
probability of observing that point if it came from cluster 0 =  0.0
probability of observing that point if it came from cluster 1 =  0.05333965685837373
point =  5.255443871801707
probability of observing that point if it came from cluster 0 =  0.0018098960884890642
probability of observing that point if it came from cluster 1 =  2.9665657346910844e-15
point =  5.285518742776113
probability of observing that point if it came from cluster 0 =  0.012747312923961645
probability of observing that point if it came from cluster 1 =  5.857221625199056e-15
point =  7.957020362100586
probability of observing that point if it came from cluster 0 =  0.0
probability of observing that point if it came from cluster 1 =  0.8701813768698816
point =  7.19475829206822
probabilit

f) Having computed `P(C_j | X_i)`, update the estimates of `mean_j`, `var_j`, and `P(C_j)`. How different are these values from the original ones you got from K means? briefly comment.

In [23]:
prob_c = [sum(prob_c0_x)/ len(prob_c0_x), sum(prob_c1_x)/ len(prob_c1_x) ]
mean = [sum([x[0] * x[1] for x in zip(prob_c0_x, data)]) / sum(prob_c0_x), sum([x[0] * x[1] for x in zip(prob_c1_x, data)]) / sum(prob_c1_x) ]
var = [ sum(map(lambda x : (x - mean[0])**2, c1)) / len(c1) , sum(map(lambda x : (x - mean[1])**2, c2)) / len(c2)  ]

print("P(C_1) = " + str(prob_c[0]) + ",  P(C_2) = " + str(prob_c[1]))
print("mean_1 = " + str(mean[0]) + ",  mean_2 = " + str(mean[1]))
print("var_1 = " + str(var[0]) + ",  var_2 = " + str(var[1]))



P(C_1) = 0.3987239738546028,  P(C_2) = 0.6012760261453972
mean_1 = 5.491945967770515,  mean_2 = 8.202110777402458
var_1 = 0.05856422063803936,  var_2 = 0.3603430653153983


variance is much bigger than the original one. Probability and mean are similar to the original ones.

g) Update `P(C_j | X_i)`. Comment on any differences or lack thereof you observe.

In [24]:
prob_c0_x = [] # P(C_j | X_i) = P(X_i | C_j) * P(C_j) / P(X_i)
prob_c1_x = [] # P(C_1 | X_i)
prob_x = [] # P(X_i) P(X_i) = sum_j P(X_i | C_j) * P(C_j)

k = 2

for p in data:
    print("point = ", p)
    pdf_i = []

    for j in range(k):
        # P(X_i | C_j)
        pdf_i.append(norm.pdf(p, mean[j], var[j]))
        print("probability of observing that point if it came from cluster " + str(j) + " = ", pdf_i[j])
        # P(C_j) already computed
        prob_c[j]

    # P(X_i) = P(C_0)P(X_i | C_0) + P(C_1)P(X_i | C_1)
    prob_x = prob_c[0] * pdf_i[0] + prob_c[1] * pdf_i[1]

    # P(C_j | X_i) = P(X_i | C_j)P(C_j) / P(X_i)
    prob_c0_x.append( pdf_i[0] * prob_c[0] / prob_x)
    prob_c1_x.append( pdf_i[1] * prob_c[1] / prob_x)

probs = zip(data, prob_c0_x, prob_c1_x)
for p in probs:
    print(p[0])
    print("Probability of coming from C_1 = " + str(p[1]))
    print("Probability of coming from C_2 = " + str(p[2]))
    print()


point =  5.849085909039437
probability of observing that point if it came from cluster 0 =  5.725763608831103e-08
probability of observing that point if it came from cluster 1 =  6.094820250557766e-10
point =  9.094537366491185
probability of observing that point if it came from cluster 0 =  0.0
probability of observing that point if it came from cluster 1 =  0.05155943162121205
point =  5.255443871801707
probability of observing that point if it came from cluster 0 =  0.001958860011806504
probability of observing that point if it came from cluster 1 =  3.338953480419549e-15
point =  5.285518742776113
probability of observing that point if it came from cluster 0 =  0.013658157297756594
probability of observing that point if it came from cluster 1 =  6.584209092372191e-15
point =  7.957020362100586
probability of observing that point if it came from cluster 0 =  0.0
probability of observing that point if it came from cluster 1 =  0.8784927581353398
point =  7.19475829206822
probability 

Comparing these estimates to the true parameter values we have from the GMM, we can see that they are somewhat close but not exact. The mixing coefficients estimated by K-means are flipped but the values are close to the true values. The covariance matrix estimates from K-means are further off, with variances that are much smaller than the true variances of 1, and the variances are also different for the two components. Overall, using K-means to estimate the initial parameters is a useful strategy but not a guarantee that the estimates will be close to the true values.

h) Use `P(C_j | X_i)` to create a hard assignment - label each point as belonging to a specific cluster (0 or 1)

In [27]:
clusters = []
for i in range(len(data)):
    print(data[i])
    if prob_c0_x[i] > prob_c1_x[i]:
        print("Point {} comes from cluster 0".format(i))
    elif prob_c0_x[i] < prob_c1_x[i]:
        print("Point {} comes from cluster 1".format(i))

5.849085909039437
Point 0 comes from cluster 0
9.094537366491185
Point 1 comes from cluster 1
5.255443871801707
Point 2 comes from cluster 0
5.285518742776113
Point 3 comes from cluster 0
7.957020362100586
Point 4 comes from cluster 1
7.19475829206822
Point 5 comes from cluster 1
5.58229254648858
Point 6 comes from cluster 0
8.615242190106898
Point 7 comes from cluster 1
7.94575907464858
Point 8 comes from cluster 1
8.435372591529761
Point 9 comes from cluster 1


### Clustering Aggregation

| Point | C | P |
|-------|---|---|
| A     | 0 | a |
| B     | 0 | b |
| C     | 2 | b |
| D     | 1 | c |
| E     | 1 | d |

a) Fill in the following table where for each pair of points determine whether C and P agree or disagree on how to cluster that pair.

| Pair | Disagreement |
|------|--------------|
| A  B |      1       |
| A  C |      0       |
| A  D |      0       |
| A  E |      0       |
| B  C |      1       |
| B  D |      0       |
| B  E |      0       |
| C  D |      0       |
| C  E |      0       |
| D  E |      1       |


As datasets become very large, this process can become computationally challenging.

b) Given N points, what is the formula for the number of unique pairs of points one can create?

(N * (N-1)) / 2

Assume that clustering C clusters all points in the same cluster and clustering P clusters points as such:

| Point | P |
|-------|---|
| A     | 0 |
| B     | 0 |
| C     | 0 |
| D     | 1 |
| E     | 1 |
| F     | 2 |
| G     | 2 |
| H     | 2 |
| I     | 2 |

c) What is the maximum number of disagreements there could be for a dataset of this size? (use the formula from b)?

Using the table of points and cluster assignments provided, we can count the number of unique pairs of points for each cluster separately:

Cluster 0: There are 3 points (A, B, and C) in this cluster, so the number of unique pairs is:

(3 choose 2) = 3(2)/2 = 3

Cluster 1: There are 2 points (D and E) in this cluster, so the number of unique pairs is:

(2 choose 2) = 2(1)/2 = 1

Cluster 2: There are 4 points (F, G, H, and I) in this cluster, so the number of unique pairs is:

(4 choose 2) = 4(3)/2 = 6

Therefore, the total number of unique pairs of points that can be created from these clusters is:

3 + 1 + 6 = 10

d) If we look at cluster 0. There are (3 x 2) / 2 = 3 pairs that agree with C (since all points in C are in the same cluster). For each cluster, determine how many agreements there are. How many total agreements are there? How many disagreements does that mean there are between C and P?

In cluster 0, all points are in the same cluster according to clustering algorithm C. Therefore, there are (3 choose 2) = (3 x 2) / 2 = 3 pairs of points that agree with C.

For each cluster, we can count the number of pairs of points that agree with C and P:

Cluster 0: 3 agreements with C, 3 agreements with P
Cluster 1: 1 agreement with C, 1 agreement with P
Cluster 2: 6 agreements with C, 6 agreements with P
To find the total number of agreements, we simply sum up the number of agreements for each cluster:

3 + 1 + 6 = 10

To find the number of disagreements, we can subtract the number of agreements from the total number of pairs of points, which we calculated in a previous answer to be:

10 (total number of pairs) - 10 (total number of agreements) = 0

Therefore, there are no disagreements between C and P in this case. This means that C and P agree on how to cluster all pairs of points.

e) Assuming that filtering the dataset by cluster number is a computationally easy operation, describe an algorithm inspired by the above process that can efficiently compute disagreement distances on large datasets.

- Divide the dataset into clusters using two different clustering algorithms (e.g., C and P).
- For each pair of clusters, compute the number of pairs of points that agree with both algorithms (i.e., the number of agreements).
- Calculate the total number of agreements by summing up the number of agreements for each pair of clusters.
- Calculate the total number of pairs of points in the dataset using the formula (N choose 2) = N(N-1)/2, where N is the total number of points in the dataset.
- Calculate the number of disagreements by subtracting the total number of agreements from the total number of pairs of points.
- Compute the disagreement distance as the ratio of the number of disagreements to the total number of pairs of points.